# Making a data set for Oil and Gas Production in Montana

## Task
Combine and reformat 2 large files of Well information and Well production.

## Technical issues encountered
- Loading large files into Colab
- Handling "tab" delimited data
- Renaming columns
- Summarizing monthly data to years 
- "pivotting" data to yearly columns
- merging 2 data sets.

## Result
The product is a CSV file that is in a format needed at FracTracker.



## Reading files
Two files are needed: 
- the first is the raw production values and is stored in a zip file on [Montana servers](http://www.bogc.dnrc.mt.gov/production/).  The code below will download that zipfile and extract the needed data.
- the second is the well information and **you** will need to gather it from the 

In [ ]:
import pandas as pd
import zipfile 
import requests
import shutil

### File 1: Download production zip file from Montana server
Downloading this zipfile (over 140Mbyte) can take **two minutes or more** due to an apparently slow server in Montana.

In [ ]:
# define an efficient routine to fetch a large file from a web address and save it
# https://stackoverflow.com/a/39217788/6736072

def download_file(url):
    local_filename = url.split('/')[-1] # just names the local file like the last part of the link
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    print(f'Downloaded {local_filename}')

In [ ]:
zfn = 'http://www.bogc.dnrc.mt.gov/production/historical.zip'
download_file(zfn)

In [ ]:
# now pull the well production file into a dataframe
# Note that in these "read_csv" functions, we set "sep" to  " \t " which is a TAB character.

with zipfile.ZipFile(zfn.split('/')[-1]) as z:
    with z.open('histprodwell.tab') as f:
        prod = pd.read_csv(f,sep='\t',  # it is TAB delimited, not comma delimited
                           low_memory=False,
                           dtype={'API_WELLNO':'str'}) # we need to treat as a string not a number

prod['year'] = pd.to_datetime(prod.rpt_date).dt.year # keep just the year

# Keep only fields that we need. 
prod = prod[['API_WELLNO','BBLS_OIL_COND', 'MCF_GAS', 'BBLS_WTR', 'DAYS_PROD', 'year']]
# rename them to something more useful
prod.columns = ['APINumber','Oil', 'Gas', 'Water','Days','year']


In [ ]:
#  Here is where we summarize by well and year
gb = prod.groupby(['APINumber','year'],as_index=False)[['Oil', 'Gas', 
                                                        'Water','Days']].sum()
gb.tail(10)

In [ ]:
# The dataframe created above is "long":  each record has only one year 
# the code code below makes it "wide"; there is only one record per well ('APINumber')

colnames = ['Oil','Gas','Water','Days']
concat_list = []
for col in colnames:
    piv = gb.pivot(index='APINumber',columns='year',values=col).fillna(0)
    names = piv.columns.tolist()
    ncols = []
    for name in names:
        ncols.append(col+'_'+str(name))
    piv.columns = ncols
    piv[f'{col}_Total'] = piv.sum(axis=1)
    concat_list.append(piv)

whole = pd.concat(concat_list,axis=1)
whole.head()

In [ ]:
piv.sum(axis=1)


## File 2: Getting the well location data and uploading it to Colab
This process takes a few manual steps.

#### Step 1: Navigate to the Montana website
Go to [this site](http://www.bogc.dnrc.mt.gov/webapps/dataminer/Wells/WellSurfaceLongLat.aspx).
It should look something like:



<img src="images/montana_1.png" height=100 />

#### Step 2: Set up search for all wells
1. Select "API #" in the first dropdown menu
2. Type in "25" in the text box
3. Click the search button

<img src="images/montana_2.png" height=100 />


**The results should look something like this:**


<img src="images/montana_3.png" height=100 />



#### Step 3: Save the results to your computer
1. Click on the "Text" button in the upper right of the screen

This will cause the website to display another page with LOTS of text.  It is actually a text file that is "tab" delimited (that is, the tab character separates the values in each row).

2. Save the file to your computer.  Usually that means doing something simple like **Ctrl-s** and following the prompts. Save it with the file name "Location.csv".  (The code below uses that name. Change the code below if you want to name it something else.  Note that in Colab, names are case-sensitive.)

#### Step 4: Move the file to Colab
1. Back in the Colab window, open the "Files" panel on the far left by clicking on the folder icon (it may already be open).
2. Click on the "upload" icon and follow the prompts to upload the file you saved in the last step.

<img src="images/montana_4.png" height=10 />

**We can now clean up this file and merge it with the production file**

## Read the Location file into a dataframe

In [ ]:
# the location data comes directly from the commission's public website:
# but note that we have saved the data using their "text" button and saved the file with a CSV extention.
#   The data are TAB delimited.
loc = pd.read_csv("Location.csv",sep = '\t')
loc.head()

## Cleanup names, merge, and save!

In [ ]:
loc['APINumber'] = loc['API #'].str.replace('-','')
loc.rename({'Wh_Long':'Longitude','Wh_Lat':'Latitude'},axis=1,inplace=True)
out = pd.merge(loc[['APINumber','Longitude', 'Latitude']],whole,
               on='APINumber',how='right',validate='1:1')

out.to_csv('MT_prod_summary.csv',index=False)

## Spot check
compare the pivot output table to the initial file (using sums across years) for a small subset of the data.  If something is screwy, this probably won't pass without errors.


Takes about one minute to run.


In [ ]:
sample = out.sample(n=20)  # get a random sample of the output dataframe
#print(sample.head())
types = ['Oil','Gas','Water','Days']
for i,row in sample.iterrows():
    api = row.APINumber
    for ty in types:
        initial = prod[prod.APINumber==api][ty].sum()
        if row[f'{ty}_Total'] != initial:
            print(f'Error in {ty} of {api}: {initial} vs. {row[f"{ty}_Total"]}')
print('Spot check completed.')